In [1]:
import tensorflow as tf
import numpy as np
import math
import sys
import os
print('Done.')

Done.


In [3]:
# Input: parametrized heights
# Output: volume

def placeholder_inputs(batch_size, num_param):
    param_pl = tf.placeholder(name="input_param", dtype=tf.float32, shape=(batch_size, num_param))
    vol_pl = tf.placeholder(name="input_vol", dtype=tf.float32, shape=(batch_size))
    
    return param_pl, vol_pl

def load_input(train_path):
    train = open(train_path, 'r')
    train_x = []
    train_y = []
    for line in train:
        data = eval(line)
        if data[1] > 0:
            train_x.append(data[0])
            inv_vol = 1/data[1]
            train_y.append(inv_vol)
    print ('Number of data: ', len(train_x))
    
    return train_x, train_y

def fl_linear(param, batch_size, num_param):
    with tf.variable_scope("fl_linear", reuse=True):
        input_param = tf.placeholder(name="input_param", dtype=tf.float32, shape=(batch_size, num_param))
        input_vol = tf.placeholder(name="input_vol", dtype=tf.float32, shape=(batch_size))
        # Linear activation
        net = tf.fully_connected(1, activation_fn=None, scope='fc1')
    
    return net

def mat_mul(param, batch_size, num_param):
    with tf.variable_scope("mat_mul", reuse=True):
        input_parameter = tf.placeholder(name="input_parameter", dtype=tf.float32, shape=(batch_size, num_param))
        input_volume = tf.placeholder(name="input_volume", dtype=tf.float32, shape=(batch_size))
        W = tf.Variable(tf.random_normal([num_param, 1], dtype=tf.float32), name="W")
        b = tf.Variable(tf.zeros([batch_size, 1], dtype=tf.float32), name="b")
        y = tf.add(tf.matmul(input_parameter, W), b)
        cost_op = tf.reduce_mean(tf.pow(y-input_volume, 2))

    return y, cost_op, input_parameter, input_volume

def train(data, vol):
    
    input_data = data[0:1000]
    input_vol = vol[0:1000]
    
    test_data = data[1000:2000]
    test_vol = vol[1000:2000]
    
    num_data = len(input_data)
    num_param = len(input_data[0])
    num_test = len(test_data)
    print('number of data: ', num_data)
    print('number of parameters: ', num_param)
    print('number of test: ', num_test)
    
    num_batches = int(num_data/batch_size)
    test_batches = int(num_test/batch_size)
    input_param = np.array(input_data)
    input_vol = np.array(input_vol)
    test_param = np.array(test_data)
    test_vol = np.array(test_vol)
    
    #y = fl_linear(input_param)
    y, cost_op, input_parameter, input_volume = mat_mul(input_param, batch_size, num_param)
    
    learning_rate = tf.Variable(0.5, trainable=False)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_op = optimizer.minimize(cost_op)
    
    sess = tf.Session() # Create TensorFlow session
    print ("Beginning Training")
    with sess.as_default():
        init = tf.global_variables_initializer()
        sess.run(init)
        sess.run(tf.assign(learning_rate, alpha))
        for epoch in range(1, max_epochs):
            print('epoch: ', epoch)
            # Train one epoch
            train_one_epoch(sess, cost_op, train_op, num_batches, input_parameter, input_param, input_volume, input_vol)
            eval_one_epoch(sess, cost_op, test_batches, input_parameter, test_param, input_volume, test_vol)
            
        print('Done.')
        W = tf.get_variable("W")
        print("Weight: ", W.eval())
            

def train_one_epoch(sess, cost_op, train_op, num_batches, input_parameter, input_param, input_volume, input_vol):
     for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = (batch_idx+1) * batch_size
        cost,_ = sess.run([cost_op, train_op], feed_dict={input_parameter: input_param[start_idx:end_idx], input_volume: input_vol[start_idx:end_idx]})
        print("loss: ", cost)
        
def eval_one_epoch(sess, cost_op, test_batches, input_parameter, test_param, input_volume, test_vol):
    for batch_idx in range(test_batches):
        start_idx = batch_idx * batch_size
        end_idx = (batch_idx+1) * batch_size
        cost = sess.run(cost_op, feed_dict={input_parameter: test_param[start_idx:end_idx], input_volume: test_vol[start_idx:end_idx]})
        print("test loss: ", cost)

# Parameters:
# last_cost = 0
# alpha = 0.4
# batch_size = 10
# max_epochs = 100
# tolerance = 1e-3

    
# train_path = '/home/carnd/CYML/output/train/cylinder/tri_1_to_50_2.txt'
# train_data, train_vol = load_input(train_path)
# print ('sample data: ', train_data[0])
# train(train_data, train_vol)

In [9]:
data = np.arange(6).reshape(2,3)
param = np.array([1,1,1])
input_data = tf.placeholder(name="input_data", dtype=tf.float32, shape=(2, 3))
input_param = tf.placeholder(name="input_param", dtype=tf.float32, shape=(1,3))
for i in range(2):
    row = input_data[i,]
    print(row)
    tmp = tf.concat([row, input_param], 1)
    try:
        out = tf.stack([input_param, tmp])
    except:
        out = tmp
        
sess = tf.Session() # Create TensorFlow session
saver = tf.train.Saver()
with sess.as_default():
    init = tf.global_variables_initializer()
    sess.run(init)
    print("slice: ", sess.run(row, feed_dict = {input_data:data, input_param:param}))
    print("stack result: ", sess.run(out, feed_dict = {input_data:data, input_param:param}))

Tensor("strided_slice_5:0", shape=(3,), dtype=float32)


ValueError: Shape must be rank 1 but is rank 2 for 'concat_5' (op: 'ConcatV2') with input shapes: [3], [1,3], [].

In [15]:
sess = tf.Session()
x = tf.constant([[1,2,3],[8,9,10]])
print(x)
#y = tf.transpose(x[0,:])
x_0 = x[0,:]
x_0 = tf.reshape(x_0,[1,-1])
x_1 = x[1,:]
x_1 = tf.reshape(x_1,[1,-1])
print(x_0)
print(x_0.eval(session=sess).tolist())
z = tf.concat([x_0,x_1],1)
print(z)
print(z.eval(session=sess).tolist())
mul = tf.matmul(tf.transpose(x_0), x_0)
print(mul)
print(mul.eval(session=sess))
for i in range(3):
    concat = tf.reshape(mul[i,i:],[1,-1])
    mul_tmp = tf.reshape(mul[i,i:],[1,-1])
    concat = tf.concat([])

Tensor("Const_12:0", shape=(2, 3), dtype=int32)
Tensor("Reshape_14:0", shape=(1, 3), dtype=int32)
[[1, 2, 3]]
Tensor("concat_12:0", shape=(1, 6), dtype=int32)
[[1, 2, 3, 8, 9, 10]]
Tensor("MatMul_4:0", shape=(3, 3), dtype=int32)
[[1 2 3]
 [2 4 6]
 [3 6 9]]


In [119]:
sess= tf.Session()
degree = 2
x = tf.constant([[1,2,3],[7,8,9]])
num_batches = 2
num_param = 3
param_fin = tf.constant([])

for i in range(num_batches):
    num_row = num_param
    x_0 = x[i, :]
    param_list = tf.reshape(x_0, [1, -1])
#     print ('param_list', param_list.eval(session=sess))
    x_1 = tf.reshape(x_0, [-1, 1])
#     print ('x_1: ', x_1.eval(session=sess))
    
    x_2 = tf.reshape(x_1, [1, -1])
    
    for deg in range(1, degree+1):
        x_3 = tf.matmul(x_1, x_2)
        print ('x_3: ', x_3.eval(session=sess))
        param_new = tf.constant([])
#         param_new[num_row] = x_3[num_row, num_row
        counter = 0
        for j in range(num_param):
            for k in range(num_param):
                if k >= j:
                    diag = x_3[j,k]
                    diag = tf.reshape(diag, [-1])
                    counter += 1
                    try:
                        param_new = tf.concat([param_new, diag[None, :]], 0)
                    except:
                        param_new = tf.reshape(diag, [-1, 1])
#                     print('counter: ', counter)
#                     print('param_new: ', param_new.eval(session=sess))
                    param_new = tf.reshape(param_new, [counter, -1])
        #             print ('concating: ', param_new.eval(session=sess))
#         diag = x_3[num_row-1,num_param-1]
        param_new = tf.reshape(param_new, [1, -1])
#         print ('param_new: ', param_new.eval(session=sess))
        for j in range(num_param, num_row):
            diag = x_3[j,:]
            diag = tf.reshape(diag, [-1])
            print('added row: ', diag)
            print('param_new: ', param_new)
            param_new = tf.concat([param_new, diag[None, :]], 1)
            print('param_new: ', param_new.eval(session=sess))
            counter += num_param
            param_new = tf.reshape(param_new, [counter, -1])
#             for k in range(num_param):
#                 diag = x_3[j,k]
#                 diag = tf.reshape(diag, [-1])
#                 print('diag')
#                 counter += 1
#                 param_new = tf.concat([param_new, diag[None, :]], 0)
#                 param_new = tf.reshape(param_new, [counter, -1])
#         diag = tf.reshape(diag, [-1])
#         print('last: ', diag.eval(session=sess))
#         param_new = tf.concat([param_new, diag[None, :]], 0)
#         print ('param_new: ', param_new.eval(session=sess))
        param_new = tf.reshape(param_new, [-1])
        print ('param_new reshaped 2: ', param_new)
        print ('param list: ', param_list)
        param_list = tf.concat([param_list, param_new[None,:]], 1)
        x_1 = tf.reshape(param_new, [-1, 1])
#         print ('x_1: ', x_1.eval(session=sess))
        num_row += 1
        
    try:
        param_list = tf.reshape(param_list, [-1])
        param_fin = tf.concat([param_fin, param_list[None, :]], 0)
    except:
        param_fin = tf.reshape(param_list, [1, -1])
    print ('param_fin:\n ', param_fin.eval(session=sess))
        

x_3:  [[1 2 3]
 [2 4 6]
 [3 6 9]]
param_new reshaped 2:  Tensor("Reshape_1826:0", shape=(6,), dtype=int32)
param list:  Tensor("Reshape_1809:0", shape=(1, 3), dtype=int32)
x_3:  [[ 1  2  3]
 [ 2  4  6]
 [ 3  6  9]
 [ 4  8 12]
 [ 6 12 18]
 [ 9 18 27]]
added row:  Tensor("Reshape_1842:0", shape=(3,), dtype=int32)
param_new:  Tensor("Reshape_1841:0", shape=(1, 6), dtype=int32)
param_new:  [[ 1  2  3  4  6  9  4  8 12]]
param_new reshaped 2:  Tensor("Reshape_1844:0", shape=(9,), dtype=int32)
param list:  Tensor("concat_765:0", shape=(1, 9), dtype=int32)
param_fin:
  [[ 1  2  3  1  2  3  4  6  9  1  2  3  4  6  9  4  8 12]]
x_3:  [[49 56 63]
 [56 64 72]
 [63 72 81]]
param_new reshaped 2:  Tensor("Reshape_1865:0", shape=(6,), dtype=int32)
param list:  Tensor("Reshape_1848:0", shape=(1, 3), dtype=int32)
x_3:  [[343 392 441]
 [392 448 504]
 [441 504 567]
 [448 512 576]
 [504 576 648]
 [567 648 729]]
added row:  Tensor("Reshape_1881:0", shape=(3,), dtype=int32)
param_new:  Tensor("Reshape_1880:

In [7]:
import tensorflow as tf
import numpy as np
import math
import sys
import os
print('Done.')

def expand_2(input_param_1, batch_size, num_param):
    for row in range(batch_size):
        param_tmp = input_param_1[row,:]
        param_tmp = tf.reshape(param_tmp, [1, -1])
        param_mat = tf.matmul(tf.transpose(param_tmp),param_tmp)

        for i in range(num_param):
            param_tmp2 = tf.reshape(param_mat[i,i:], [1,-1])
            #print(param_tmp2.eval(session=sess))
            param_tmp = tf.concat([param_tmp,param_tmp2], 1)
        param_tmp = tf.reshape(param_tmp, [-1])

        try:
            input_parameter = tf.concat([input_parameter,param_tmp[None, :]],0)
            input_parameter = tf.reshape(input_parameter, [row+1, -1])
        except:
            input_parameter = param_tmp
            input_parameter = tf.reshape(input_parameter, [row+1, -1])
    
    return input_parameter

sess = tf.Session()
data = np.arange(18)
data = data.reshape([6,-1])
batch_size = 6
num_param = 3
input_param_1 = tf.constant(data, dtype=tf.float32)
input_parameter = expand_2(input_param_1, batch_size, num_param)

# for row in range(batch_size):
#     param_tmp = input_param_1[row,:]
#     param_tmp = tf.reshape(param_tmp, [1, -1])
#     param_mat = tf.matmul(tf.transpose(param_tmp),param_tmp)

#     for i in range(num_param):
#         param_tmp2 = tf.reshape(param_mat[i,i:], [1,-1])
#         #print(param_tmp2.eval(session=sess))
#         param_tmp = tf.concat([param_tmp,param_tmp2], 1)
#     param_tmp = tf.reshape(param_tmp, [-1])

#     try:
#         input_parameter = tf.concat([input_parameter,param_tmp[None, :]],0)
#         input_parameter = tf.reshape(input_parameter, [row+1, -1])
#     except:
#         input_parameter = param_tmp
#         input_parameter = tf.reshape(input_parameter, [row+1, -1])

print(input_parameter.eval(session=sess))
print(tf.shape(input_parameter).eval(session=sess))

Done.
[[  0.   1.   2.   0.   0.   0.   1.   2.   4.]
 [  3.   4.   5.   9.  12.  15.  16.  20.  25.]
 [  6.   7.   8.  36.  42.  48.  49.  56.  64.]
 [  9.  10.  11.  81.  90.  99. 100. 110. 121.]
 [ 12.  13.  14. 144. 156. 168. 169. 182. 196.]
 [ 15.  16.  17. 225. 240. 255. 256. 272. 289.]]
[6 9]


In [13]:
def load_input(train_path):
    train = open(train_path, 'r')
    train_x = []
    train_y = []
    counter = 0
    for line in train:
        if counter > 10:
            break
        data = eval(line)
        if data[1] > 0:
            train_x.append(data[0])
#             inv_vol = 1/data[1]
            train_y.append(data[1])
        counter += 1
    print ('Number of data: ', len(train_x))
    
    return train_x, train_y

In [39]:
W = np.array([-0.87437505,
             6.952491,
              4.5137253,
              0.74184924,
              2.0514772 ,
             -0.14970087,
             -0.13637641,
             -0.20277256,
             -0.05430971])
b = np.array([-35.561787,
             -35.39475 ,
             -35.3224  ,
             -34.635918,
             -34.72557 ,
             -34.767803,
             -34.995155,
             -34.83515 ,
             -34.908066,
             -34.896793])
W = np.array([-1.32813477e+03,
              2.83764465e+02,
              5.81152466e+02,
              1.65513077e+02,
              1.64283314e+01,
             -1.20188210e+02,
              1.55702343e+01,
              1.09391075e+02,
             -1.57943008e+02,
             -4.78923941e+00,
             -2.92627001e+00,
              1.90355957e+00,
             -5.39966488e+00,
             -4.67496252e+00,
              1.12794566e+00,
              7.61268473e+00,
              1.16817160e+01,
              6.08279085e+00,
             -1.99237764e+00])
b = np.array([-811.7567 
             -751.0992 ,
             -827.5288 ,
             -879.7452 ,
             -963.4682 ,
             -740.0352 ,
             -635.4922 ,
             -712.0758 ,
             -713.21295,
             -757.816  ])

# Mean:
b_mean = np.mean(b)
print('mean: ', b_mean)

# Test if this model actually works
train_path = '/home/carnd/CYML/output/train/cylinder/lift_1_to_50.txt'
train_data, train_vol = load_input(train_path)
num_data = len(train_data)
num_param = len(train_data[0])
# transform the num_data

for i in range(num_data):
    data = np.array(train_data[i])
    data = np.expand_dims(data, axis=1)
#     print('original: ', data)
    data2 = data.reshape([num_param]).tolist()
#     print('reshaped: ',data2)
    data = np.matmul(data, np.transpose(data))
#     print('matrix: ', data)
    for j in range(num_param):
        # for each row of this matrix
        data2 += data[j,j:].tolist()
        #data2.append(data[j,j:].tolist())
#     print('extended data: ', data2)
    pred = np.dot(data2, W)
    diff = abs(1/float(pred) - train_vol[i])
    print('diff', diff)
    

mean:  -35.0043392
Number of data:  11
diff 0.5022808532687777
diff 0.5022808532542258
diff 0.327472965164726
diff 0.5022808533524512
diff 0.32747296516472557
diff 0.2494767435816346
diff 0.23791506801962586
diff 0.5022808532396739
diff 0.3274729651647258
diff 0.24947674356162572
diff 0.23791506801962442


In [26]:
def load_input(train_path):
    train = open(train_path, 'r')
    train_x = []
    train_y = []
    counter = 0
    for line in train:
        if counter >= 10:
            break
        data = eval(line)
        if data[1] > 0:
            train_x.append(data[0])
            inv_vol = 1/data[1]
            train_y.append(inv_vol)
        counter += 1
    print ('Number of data: ', len(train_x))
    
    return train_x, train_y

def expand_3(input_param_1, W, b_m, train_vol):
    # TODO: make this more general
    num_param = 3
    input_param = 10
    for row in range(batch_size):
        param_tmp = input_param_1[row,:]
        param_tmp = tf.reshape(param_tmp, [1, -1])
        p1 = param_tmp[0,0]
        p2 = param_tmp[0,1]
        p3 = param_tmp[0,2]
        
        p = tf.pow(p1,2)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        p = tf.pow(p2,2)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        p = tf.pow(p3,2)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        
        p = tf.multiply(p1, p2)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        p = tf.multiply(p1, p3)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        p = tf.multiply(p2, p3)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        
        p = tf.pow(p1,3)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        p = tf.pow(p2,3)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        p = tf.pow(p3,3)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        
        p = tf.multiply(tf.pow(p1,2), p2)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        p = tf.multiply(tf.pow(p1,2), p3)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        p = tf.multiply(p1, tf.pow(p2,2))
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        p = tf.multiply(p1, tf.pow(p3,2))
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        
        p = tf.multiply(tf.pow(p2,2), p3)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        p = tf.multiply(tf.pow(p3,2), p2)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        
        p = tf.multiply(tf.multiply(p1, p2),p3)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        param_tmp = tf.cast(param_tmp, tf.float64)
        
        try:
            param_tmp = tf.reshape(param_tmp, [-1])
            input_param = tf.concat([input_param, param_tmp[None,:]], 0)
        except:
            input_param = tf.reshape(param_tmp, [1, -1])
    
    W = tf.constant(W)
    W = tf.reshape(W, [-1,1])
    W = tf.cast(W, tf.float64)
    b_m = tf.constant(b_m)
    b_m = tf.cast(b_m, tf.float64)
    y = tf.add(tf.matmul(input_param, W), b_m)
    train_vol = tf.constant(train_vol, dtype=tf.float64)
    train_vol = tf.cast(train_vol, tf.float64)
    print('y: ', y.eval(session=sess))
    print('train_vol: ', train_vol.eval(session=sess))
#     cost = tf.reduce_mean(tf.pow(y-tf.multiply(train_vol, y),2))
    cost = tf.reduce_mean(tf.pow(y-train_vol,2))
    num_param = num_param + 6+ 10
    return input_param, cost

W = np.array([-1.32813477e+03,
              2.83764465e+02,
              5.81152466e+02,
              1.65513077e+02,
              1.64283314e+01,
             -1.20188210e+02,
              1.55702343e+01,
              1.09391075e+02,
             -1.57943008e+02,
             -4.78923941e+00,
             -2.92627001e+00,
              1.90355957e+00,
             -5.39966488e+00,
             -4.67496252e+00,
              1.12794566e+00,
              7.61268473e+00,
              1.16817160e+01,
              6.08279085e+00,
             -1.99237764e+00])
b = np.array([-811.7567 
             -751.0992 ,
             -827.5288 ,
             -879.7452 ,
             -963.4682 ,
             -740.0352 ,
             -635.4922 ,
             -712.0758 ,
             -713.21295,
             -757.816  ])

sess = tf.Session()
train_path = '/home/carnd/CYML/output/train/cylinder/lift_1_to_50.txt'
train_data, train_vol = load_input(train_path)
print('train_data: ', train_data)
print('train_vol: ', train_vol)
input_data = tf.constant(train_data)
b = b.reshape([-1,1])
b_mean = np.mean(b)
input_param, cost = expand_3(input_data, W, b_mean, train_vol)
print('cost: ', cost.eval(session=sess))
input_data = input_param.eval(session=sess)
# print('input_param: ', input_data)
W = W.reshape([-1,1])
print('W: ', W)
pred_list = np.dot(input_data, W)




for row in range(pred_list.shape[0]):
    pred = 1/(pred_list[row]+b_mean)
    diff = abs(1/train_vol[row] - pred)
    print ('diff: ', diff)

Number of data:  10
train_data:  [[1, 0, 0], [1, 1, 0], [2, 0, 0], [1, 1, 1], [2, 1, 0], [3, 0, 0], [2, 1, 1], [2, 2, 0], [3, 1, 0], [4, 0, 0]]
train_vol:  [1.6874999999416787, 2.3703703703703702, 3.3749999998833573, 2.8854381997703142, 2.4816806884647713, 5.06249999966964, 4.417092671887906, 4.740740740300995, 2.5292143561705105, 6.749999999352326]
y:  [[-2033.21429352]
 [-1724.64925205]
 [-2898.33450839]
 [-1291.62351806]
 [-2589.2702816 ]
 [-3489.89944218]
 [-2055.25805308]
 [-2260.39522943]
 [-3191.13535983]
 [-3836.64453135]]
train_vol:  [1.6875     2.37037037 3.375      2.8854382  2.48168069 5.0625
 4.41709267 4.74074074 2.52921436 6.75      ]
cost:  7048172.7598907305
W:  [[-1.32813477e+03]
 [ 2.83764465e+02]
 [ 5.81152466e+02]
 [ 1.65513077e+02]
 [ 1.64283314e+01]
 [-1.20188210e+02]
 [ 1.55702343e+01]
 [ 1.09391075e+02]
 [-1.57943008e+02]
 [-4.78923941e+00]
 [-2.92627001e+00]
 [ 1.90355957e+00]
 [-5.39966488e+00]
 [-4.67496252e+00]
 [ 1.12794566e+00]
 [ 7.61268473e+00]
 [ 1.168

In [ ]:
def mat_mul(param, batch_size, num_param, deg):
    # Hyper-parmeter: how many layers we will need
    with tf.variable_scope("mat_mul", reuse=True):
        input_param_1 = tf.placeholder(name="input_param_1", dtype=tf.float32, shape=(batch_size, num_param))
        input_volume = tf.placeholder(name="input_volume", dtype=tf.float32, shape=(batch_size))
        
        for i in range(deg):
            deg_n1 = tf.matmul(deg_n0, tf.transpose(deg_1))
            deg_n2 = tf.concat([deg_n1, deg_1])
        
        
        
        input_parameter = expand_2(input_param_1, batch_size, num_param)
        num_param = int(num_param + num_param*(num_param+1)/2)

        W1 = tf.Variable(tf.random_normal([num_param, 1], dtype=tf.float32), name="W")
        b1 = tf.Variable(tf.zeros([batch_size, 1], dtype=tf.float32), name="b")
        y = tf.add(tf.matmul(input_parameter, W1), b1)
        
        cost_op = tf.reduce_mean(tf.pow(y-input_volume, 2))
    
    return y, cost_op, input_param_1, input_volume, input_parameter

In [145]:
def expand_3(input_param_1, batch_size, num_param):
    # TODO: make this more general
    num_param = 3
    input_param = 10
    for row in range(batch_size):
        param_tmp = input_param_1[row,:]
        param_tmp = tf.reshape(param_tmp, [1, -1])
        p1 = param_tmp[0,0]
        p2 = param_tmp[0,1]
        p3 = param_tmp[0,2]
        
        p = tf.pow(p1,2)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        p = tf.pow(p2,2)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        p = tf.pow(p3,2)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        
        p = tf.multiply(p1, p2)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        p = tf.multiply(p1, p3)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        p = tf.multiply(p2, p3)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        
        p = tf.pow(p1,3)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        p = tf.pow(p2,3)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        p = tf.pow(p3,3)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        
        p = tf.multiply(tf.pow(p1,2), p2)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        p = tf.multiply(tf.pow(p1,2), p3)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        p = tf.multiply(p1, tf.pow(p2,2))
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        p = tf.multiply(p1, tf.pow(p3,2))
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        
        p = tf.multiply(tf.pow(p2,2), p3)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        p = tf.multiply(tf.pow(p3,2), p2)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        
        p = tf.multiply(tf.multiply(p1, p2),p3)
        p = tf.reshape(p, [-1])
        param_tmp = tf.concat([param_tmp, p[None,:]], 1)
        
        print('param_tmp:', param_tmp.eval(session=sess))
        
        try:
            param_tmp = tf.reshape(param_tmp, [-1])
            input_param = tf.concat([input_param, param_tmp[None,:]], 0)
            print('concat')
        except:
            print('init')
            input_param = tf.reshape(param_tmp, [1, -1])
        print('input_param:',input_param.eval(session=sess))
    
    num_param = num_param + 6+ 10
    return input_param, num_param

sess = tf.Session()
input_param = tf.constant([[1,2,3],[2,3,10]])
input_param, num_param = expand_3(input_param, 2, 3)
print ('num_param: ', num_param)

param_tmp: [[ 1  2  3  1  4  9  2  3  6  1  8 27  2  3  4  9 12 18  6]]
init
input_param: [[ 1  2  3  1  4  9  2  3  6  1  8 27  2  3  4  9 12 18  6]]
param_tmp: [[   2    3   10    4    9  100    6   20   30    8   27 1000   12   40
    18  200   90  300   60]]
concat
input_param: [[   1    2    3    1    4    9    2    3    6    1    8   27    2    3
     4    9   12   18    6]
 [   2    3   10    4    9  100    6   20   30    8   27 1000   12   40
    18  200   90  300   60]]
num_param:  19


In [7]:
# Game plan:
# Method one: extend input paramters
# Method two: multiply variables together to obtain non-linearity

def load_input_extend(train_path):
    train = open(train_path, 'r')
    train_x = []
    train_y = []
    counter = 1
    for line in train:
        if counter > 50:
            break
        data = eval(line)
        counter += 1
        if data[1] > 0:
            p1 = data[0][0]
            p2 = data[0][1]
            p3 = data[0][2]
            p = p1**2
            data[0].append(p)
            p = p2**2
            data[0].append(p)
            p = p3**2
            data[0].append(p)
            p = p1*p2
            data[0].append(p)
            p = p1*p3
            data[0].append(p)
            p = p2*p3
            data[0].append(p)
            p = p1**3
            data[0].append(p)
            p = p2**3
            data[0].append(p)
            p = p3**3
            data[0].append(p)
            p = (p1**2)*p2
            data[0].append(p)
            p = (p1**2)*p3
            data[0].append(p)
            p = p1*(p2**2)
            data[0].append(p)
            p = p1*(p3**2)
            data[0].append(p)
            p = (p2**2)*p3
            data[0].append(p)
            p = (p3**2)*p2
            data[0].append(p)
            p = p1*p2*p3 
            data[0].append(p)
            train_x.append(data[0])
            inv_vol = 1/data[1]
            train_y.append(inv_vol)
    print ('Number of data: ', len(train_x))
    
    return train_x, train_y

def mat_mul_3(param, batch_size, num_param):
    # TODO: make this more general
    with tf.variable_scope("mat_mul_3", reuse=True):
        input_parameter = tf.placeholder(name="input_parameter", dtype=tf.float32, shape=(batch_size, num_param))
        input_volume = tf.placeholder(name="input_volume", dtype=tf.float32, shape=(batch_size))
        
        # Layer 1:
        # Input: (batch_size, num_param)
        # W1: (num_param, 10), b: (batch_size, 10)
        # Output: (batch_size, 10)
        W1 = tf.Variable(tf.random_normal([num_param, 10], dtype=tf.float32), name="W1")
        y = tf.matmul(input_parameter, W1)
        layer_1 = y
        
        # Layer 2:
        # Input: (batch_size, 10)
        # W1: (10, 5), b: (batch_size, 5)
        # Output: (batch_size, 5)
        W2 = tf.Variable(tf.random_normal([10, 5], dtype=tf.float32), name="W2")
        y = tf.matmul(y, W2)
        layer_2 = y
        
        # Layer 3:
        # Input: (batch_size, 5)
        # W1: (5, 1), b: (batch_size, 1)
        # Output: (batch_size, 1)
        W3 = tf.Variable(tf.random_normal([5, 1], dtype=tf.float32), name="W3")
        b3 = tf.Variable(tf.zeros([batch_size, 1], dtype=tf.float32), name="b3")
        y = tf.add(tf.matmul(y, W3), b3)
        layer_3 = y
        
        #cost_op = tf.reduce_mean(tf.pow(tf.pow(y,-1)-tf.multiply(tf.pow(input_volume,-1), y), 2))
        cost_op = tf.reduce_mean(tf.pow(y-input_volume, 2))
        #cost_op = tf.reduce_mean(tf.multiply(tf.pow(y-input_volume, 2), tf.pow(y, -3/4)))

    return y, cost_op, input_parameter, input_volume, layer_1, layer_2, layer_3

def train(data, vol):
    num_data = len(data)
    split_idx = int(num_data*0.8)
    
    input_data = data[0:split_idx]
    input_vol = vol[0:split_idx]
    
    test_data = data[split_idx:]
    test_vol = vol[split_idx:]
    
    num_train = split_idx
    num_test = num_data - split_idx
    num_param = len(input_data[0])
    num_test = len(test_data)
    print('number of train: ', num_train)
    print('number of test: ', num_test)
    print('number of parameters: ', num_param)
    print('number of test: ', num_test)
    
    num_batches = int(num_train/batch_size)
    print('batch_size: ', batch_size)
    print('number of batches: ', num_batches)
    test_batches = int(num_test/batch_size)
    input_param = np.array(input_data)
    input_vol = np.array(input_vol)
    test_param = np.array(test_data)
    test_vol = np.array(test_vol)
    
    #y = fl_linear(input_param)
    y, cost_op, input_parameter, input_volume, layer_1, layer_2, layer_3 = mat_mul_3(input_param, batch_size, num_param)
    
    learning_rate = tf.Variable(0.5, trainable=False)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_op = optimizer.minimize(cost_op)
    
    sess = tf.Session() # Create TensorFlow session
    saver = tf.train.Saver() # Save Tensorflow graph
    print ("Beginning Training")
    with sess.as_default():
        init = tf.global_variables_initializer()
        sess.run(init)
        sess.run(tf.assign(learning_rate, alpha))
        for epoch in range(1, max_epochs):
            print('epoch: ', epoch)
            train_one_epoch(sess, cost_op, train_op, num_batches, input_parameter, input_param, input_volume, input_vol, y, layer_1, layer_2, layer_3)
            eval_one_epoch(sess, cost_op, test_batches, input_parameter, test_param, input_volume, test_vol)
            
            if epoch % 10 == 0:
                path = "log/model_3.ckpt"
                save_path = saver.save(sess, path)
                print("Saved.")
            
        print('Done.')
        
        collection = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='mat_mul_3')
        for x in collection:
            if x.name == 'mat_mul_3/W1:0' or x.name == 'mat_mul_3/W2:0'or x.name == 'mat_mul_3/W3:0' or x.name == 'mat_mul_3/b3:0':
                print("name: ", x.name)
                print("variables: ", sess.run(x))
                
def train_one_epoch(sess, cost_op, train_op, num_batches, input_parameter, input_param, input_volume, input_vol, y, layer_1, layer_2, layer_3):
    print('batches: ', num_batches)
    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = (batch_idx+1) * batch_size

        batch_cost,_, batch_layer_1, batch_layer_2, batch_layer_3 = sess.run([cost_op, train_op, layer_1, layer_2, layer_3], feed_dict={input_parameter: input_param[start_idx:end_idx], input_volume: input_vol[start_idx:end_idx]})
        print("loss: ", batch_cost)
        print("layer_1: ", batch_layer_1)
        print("layer_2: ", batch_layer_2)
        print("layer_3: ", batch_layer_3)

def eval_one_epoch(sess, cost_op, test_batches, input_parameter, test_param, input_volume, test_vol):
    for batch_idx in range(test_batches):
        start_idx = batch_idx * batch_size
        end_idx = (batch_idx+1) * batch_size
        cost = sess.run(cost_op, feed_dict={input_parameter: test_param[start_idx:end_idx], input_volume: test_vol[start_idx:end_idx]})
        print("test loss: ", cost)

# Parameters:
last_cost = 0
alpha = 0.4
batch_size = 5
max_epochs = 10        

train_path = '/home/carnd/CYML/output/train/cylinder/lift_1_to_50.txt'
train_data, train_vol = load_input_extend(train_path)
print ('sample data: ', train_data[0])
train(train_data, train_vol)

Number of data:  50
sample data:  [1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
number of train:  40
number of test:  10
number of parameters:  19
number of test:  10
batch_size:  5
number of batches:  8
Beginning Training
epoch:  1
batches:  8
loss:  227.53397
layer_1:  [[ -0.08400881   0.7285836   -1.4688802    1.7337687    2.4287684
   -1.1019212    0.17918992   0.6151695    1.6486315    0.39970595]
 [  0.9857779    0.8922627   -5.389453     5.3263       2.1905851
    1.7714543   -1.3625824    1.7792897    2.3743463    2.0419292 ]
 [  2.3964784    5.4760523  -13.864322     9.682614    14.221719
  -10.777562    -2.7799118    2.513164     7.819329    -2.080552  ]
 [ -0.8367122    1.412885    -7.1304455   -1.1579419    4.2312465
    1.4391825    4.9699774   -2.546794    -0.02583212  -5.5218344 ]
 [  5.750342     4.0370774  -21.900763    13.528143    12.8580065
   -3.6056557   -5.769534     7.086888    10.055863     0.91088074]]
layer_2:  [[  1.4889588    4.190979    -6.6100

In [17]:
print ('invert: ', train_y[0]**(-1))
print ('invert: ', (train_y[0]**(-1))**(-1))

invert:  0.5925925926130731
invert:  1.6874999999416787
